# 1. Accessing the Dataset

## 1.1. Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

## 1.2. Import the Dataset

In [ ]:
url = 'https://raw.githubusercontent.com/hazardous-sun/automobile-dataset-experiments/main/assets/automobile/imports-85.data'

# 1. Define the column names
column_names = [
    'symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration',
    'num-of-doors', 'body-style', 'drive-wheels', 'engine-location',
    'wheel-base', 'length', 'width', 'height', 'curb-weight',
    'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system',
    'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm',
    'city-mpg', 'highway-mpg', 'price'
]

# 2. Read the CSV from the URL with all the correct parameters
dataset = pd.read_csv(
    url,
    header=None,         # No header in the file
    names=column_names,  # Use our defined column names
    na_values="?"        # Mark '?' as missing data
)

dataset

# 1.3. Checking how the fields were interpreted

In [ ]:
dataset.dtypes

# 1.4. Initial dataset analysis

In [ ]:
dataset.describe()

# 2. Pre-processing

## 2.1. Encodding

In [ ]:
one_hot_dataset = pd.get_dummies(dataset,drop_first=True)
one_hot_dataset

In [ ]:
for i, column in enumerate(one_hot_dataset.columns):
  print(f'{i} - {column}')

In [ ]:
X = one_hot_dataset[:15] + one_hot_dataset[16:]
X

In [ ]:
y = one_hot_dataset['price']
y

## 2.2. Separating between training and testing

- Training: 70%
- Testing: 30%

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)